In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load DialoGPT-small model and tokenizer
model_name = "microsoft/DialoGPT-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Check if GPU is available and move model to GPU if possible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Set chat parameters
max_length = 1000
chat_history_ids = None

def chat_with_bot(tkinter_chat_input):
    global chat_history_ids
    
    # Encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(tkinter_chat_input + tokenizer.eos_token, return_tensors='pt').to(device)
    
    # Create the attention mask
    new_user_attention_mask = torch.ones(new_user_input_ids.shape, dtype=torch.long, device=device)

    # Append the new user input tokens to the chat history
    if chat_history_ids is not None:
        bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1)
        attention_mask = torch.cat([torch.ones(chat_history_ids.shape, dtype=torch.long, device=device), new_user_attention_mask], dim=-1)
    else:
        bot_input_ids = new_user_input_ids
        attention_mask = new_user_attention_mask

    # Generate a response while limiting the total chat history to 1000 tokens
    chat_history_ids = model.generate(
        bot_input_ids, 
        max_length=max_length,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=3,
        do_sample=True,
        top_k=100,
        top_p=0.7,
        temperature=0.8,
        attention_mask=attention_mask
    )
    
    # Extract the AI's response
    ai_response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    
    # Optionally, trim conversation history if it gets too long
    if chat_history_ids.shape[1] > 1000:
        chat_history_ids = chat_history_ids[:, -1000:]
    
    return ai_response



c:\Users\pawan\anaconda3\envs\AAI520\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from tkinter import *
from tkinter import ttk

user_chat_history = []

def updatechat(*args):
    global user_chat_history
    
    user_input = dialog.get()    
    
    # Check for termination commands
    if user_input.lower() in ["quit", "bye"]:
        chat.set("Thank you for chatting! Goodbye!")
        dialog.set("")  # Clear the entry box
        return  # Exit the function early
    
    user_chat_history.append(user_input)
    
    if len(user_chat_history) > 1000:
        user_chat_history = user_chat_history[-1000:]
    
    # Get the AI's response
    ai_response = chat_with_bot(user_input)
    
    # Append the AI's response to the chat history
    user_chat_history.append(ai_response)

    # Update the chat_history StringVar
    chat.set("\n".join(user_chat_history))
    
    # Clear the entry field after sending the message
    dialog.set("")  # Clear the entry box

# Create the main application window
app = Tk()
app.title("AAI520 Dialog Chat")

mainframe = ttk.Frame(app, padding="50 3 20 12")
mainframe.grid(column=0, row=0, sticky=(N, W, E, S))

# Configure the grid to have 5 columns and 5 rows
for i in range(5):
    mainframe.columnconfigure(i, weight=1)  # Allow columns to expand
    mainframe.rowconfigure(i, weight=1)     # Allow rows to expand

# Row 1: Instructions
label = ttk.Label(mainframe, text="Chat with the AI. Type 'exit', 'quit', or 'end' to finish the conversation.")
label.grid(column=2, row=1, sticky=(W, E))

# Row 2: Chat History
label = ttk.Label(mainframe, text="Chat History:")
label.grid(column=1, row=2, sticky=(W, E))
chat = StringVar()
ttk.Label(mainframe, textvariable=chat).grid(column=2, row=2, sticky=(W, E))

# Row 3: Input Box
label = ttk.Label(mainframe, text="Input: ")
label.grid(column=1, row=3, sticky=(W, E))

dialog = StringVar()
dialog_entry = ttk.Entry(mainframe, width=150, textvariable=dialog)
dialog_entry.grid(column=2, row=3, sticky=(W, E))

# Chat button
ttk.Button(mainframe, text="Chat", command=updatechat).grid(column=2, row=4, sticky=(W, E))

# Add padding to all child widgets
for child in mainframe.winfo_children(): 
    child.grid_configure(padx=5, pady=5)

# Set focus to the entry field and bind the Enter key to the chat function
dialog_entry.focus()
app.bind("<Return>", updatechat)

# Run the application
app.mainloop()


c:\Users\pawan\anaconda3\envs\AAI520\Lib\site-packages\transformers\models\gpt2\modeling_gpt2.py:545: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [7]:
import gradio as gr
import random
import time

def respond(message, chat_history):
    bot_message = chat_with_bot(message)
    chat_history.append((message, bot_message))
    
    # Optionally limit chat history to a certain length
    if len(chat_history) > 10:  # Keep only the last 10 exchanges
        chat_history = chat_history[-10:]

    return "", chat_history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

if __name__ == "__main__":
    demo.launch()

Running on local URL:  http://127.0.0.1:7863

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
